<a href="https://colab.research.google.com/github/AdheFirmansyah/DataScience/blob/main/Taskupwork_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cart = { "name":"X", "mean":456, "split_by_feature": "aes", "error_of_split": 0.0,
        "successor_left": { "name":"XL", "mean":1234, "split_by_feature": None, "error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          },
        "successor_right":{ "name":"XR", "mean":258, "split_by_feature": None,"error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          }
       }

In [2]:
features = ["secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
            "timestamp", "segmentation", "onehundredmb", "onegb"]

# TASK 1 : Create CART

In [3]:
import pandas as pd

In [4]:
def get_cart(sample_set_csv):
    df = pd.read_csv(sample_set_csv)
    features = ["secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
                "timestamp", "segmentation", "onehundredmb", "onegb"]
    target = "mean"

    # Membuat pohon dengan kriteria pemisahan "mean"
    def create_tree(df, features, target):
        # Menentukan node saat ini
        current_node = {"name": None, "mean": None, "split_by_feature": None, "error_of_split": None,
                        "successor_left": None, "successor_right": None}
        current_node["mean"] = df[target].mean()

        # Jika tidak ada fitur yang tersedia atau jika semua data di node saat ini memiliki target yang sama
        if len(features) == 0 or len(set(df[target])) == 1:
            return current_node

        # Menentukan fitur dengan nilai split yang paling baik
        best_feature = None
        best_split_value = None
        best_error = float("inf")
        for feature in features:
            for value in set(df[feature]):
                left_data = df[df[feature] <= value]
                right_data = df[df[feature] > value]

                # Hitung error dari split
                error = (len(left_data) / len(df)) * (left_data[target].var() + right_data[target].var())

                # Jika error lebih rendah dari yang sebelumnya, simpan fitur dan nilai split
                if error < best_error:
                    best_feature = feature
                    best_split_value = value
                    best_error = error

        # Jika tidak ada fitur yang dapat dipakai untuk split
        if best_error == float("inf"):
            return current_node

        # Simpan fitur dan nilai split yang digunakan
        current_node["split_by_feature"] = best_feature
        current_node["error_of_split"] = best_error

        # Hapus fitur yang digunakan dari daftar fitur
        features.remove(best_feature)

        # Pisahkan data menjadi dua node
        left_data = df[df[best_feature] <= best_split_value]
        right_data = df[df[best_feature] > best_split_value]

        # Pergi ke node anak sebelah kiri dan kanan
        current_node["successor_left"] = create_tree(left_data, features, target)
        current_node["successor_right"] = create_tree(right_data, features, target)

        return current_node

def get_highest_influence_feature(cart):
    highest_influence_feature = None
    highest_influence = -1

    def explore_node(node):
        nonlocal highest_influence_feature
        nonlocal highest_influence

        if node["successor_left"] is None and node["successor_right"] is None:
            return
        else:
            # jika feature memiliki nilai influence yang lebih tinggi dari yang sebelumnya
            if node["influence"] > highest_influence:
                highest_influence = node["influence"]
                highest_influence_feature = node["split_by_feature"]
            explore_node(node["successor_left"])
            explore_node(node["successor_right"])
    explore_node(cart)
    return highest_influence_feature

    
def get_error_rate(cart, sample_set_csv):
    df = pd.read_csv(sample_set_csv)
    target = "mean"
    incorrect_predictions = 0

    for i, data in df.iterrows():
        node = cart
        while node["successor_left"] is not None and node["successor_right"] is not None:
            if data[node["split_by_feature"]] <= data[node["split_by_feature"]]:
                node = node["successor_left"]
            else:
                node = node["successor_right"]

        # Menghitung prediksi yang salah
        if data['performance'] != node["mean"]:
            incorrect_predictions += 1

    # Menghitung tingkat kesalahan dalam persentase
    error_rate = incorrect_predictions / len(df) * 100
    return error_rate


def get_optimal_configuration(cart, partial_configuration):
    optimal_config = set()
    optimal_error = float("inf")

    def explore_node(node, config):
        nonlocal optimal_config
        nonlocal optimal_error

        if node["successor_left"] is None and node["successor_right"] is None:
            # Jika node ini adalah daun dan konfigurasi saat ini lebih baik daripada yang sebelumnya ditentukan
            if node["error_of_split"] is not None and node["error_of_split"] < optimal_error:
                optimal_config = config.copy()
                optimal_error = node["error_of_split"]
        else:
            # Pergi ke node anak sebelah kiri dengan menambahkan fitur pemisah ke konfigurasi
            explore_node(node["successor_left"], config.union({node["split_by_feature"]}))

            # Pergi ke node anak sebelah kanan dengan menambahkan fitur pemisah ke konfigurasi
            explore_node(node["successor_right"], config.union({node["split_by_feature"]}))

    explore_node(cart, partial_configuration)
    return optimal_config

In [5]:
test_cart = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'segmentation', 'error_of_split': 6.0, 
             'successor_left': 
                 {'name': 'XL', 'mean': 772.0, 'split_by_feature': 'onegb', 'error_of_split': 0.0, 
                  'successor_left': 
                      {'name': 'XLL', 'mean': 770.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }, 
                  'successor_right': 
                      {'name': 'XLR', 'mean': 773.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }
                 }, 
             'successor_right': 
                 {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                  'successor_left': None, 
                  'successor_right': None}
            }


if get_cart("Performance_01.csv") == test_cart:
    print("passed")
else:
    print("failed")

failed


In [6]:
if get_error_rate(test_cart, "Performance_01.csv") == 5:
    print("passed")
else:
    print("failed")

failed


In [7]:
# Task 2c
test_cart_v2 = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'zip', 'error_of_split': 0.0, 
                 'successor_left': {'name': 'XL', 'mean': 772.0, 'split_by_feature': None, 'error_of_split': None, 
                                    'successor_left': None, 
                                    'successor_right': None}, 
                 'successor_right': {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                                     'successor_left': None, 
                                     'successor_right': None}
                }

optimal_config = get_optimal_configuration(test_cart_v2, {"secompress", "encryption", "aes", "algorithm", "signature",
                                                        "timestamp", "segmentation", "onehundredmb"})
reference = {'aes', 'algorithm', 'encryption', 'onehundredmb', 'rar', 'secompress', 'segmentation', 'signature',
            'timestamp'}

if optimal_config == reference:
    print("passed")
else:
    print("failed")

failed
